<a href="https://colab.research.google.com/github/jai-llm/TEXT2SQL/blob/main/0_Text2SQL_Create_Datav3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text2SQL Data Pre-Processing

 Process Dataset For Training & Evaluation

In [1]:
!pip install -q -U datasets
!pip install evaluate




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


### Import Libraries

In [2]:
import pandas as pd
import json
import torch
import os
# Load Methods from Datasets Library
from datasets import load_dataset, Dataset, load_from_disk
from evaluate import load as load_metric


### Global Variables

In [3]:
dataset_name = "b-mc2/sql-create-context"

In [4]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"

# Generating Train/Test Data Parameters
TABLE_NAMES = True # Drops Most of WikiSQL
SIMPLE_INST = False # Complex Prompt
SAMPLE_RATE = 0.1 # Train=90% vs Test=10% Split

In [5]:
from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


### Common Functions

In [6]:
def process_data(dataset_name, sample_rate,
                 table_names=False, simple_inst=False):
  '''Function Returns a DataFrame '''
  # 1. Move data to df
  txt2sql_ds = load_dataset(dataset_name)
  txt2sql_df = pd.DataFrame(txt2sql_ds)
  dsdf = pd.json_normalize(txt2sql_df['train'])
  # display(dsdf.head(2))

  # 2. Cleanup Steps
  # Dropping all examples where no definite table name is given
  # i.e. most of WikiSQL
  if table_names:
    dsdf = dsdf.loc[~dsdf['answer'].str.contains('FROM table_')]
  # Drop Dups
  dsdf.drop_duplicates(inplace=True)
  dsdf.rename(columns={'answer': 'response'}, inplace=True)

  # 3. Simple Instruction
  if simple_inst:
    template = """Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    Generate SQL query: {question}, \
    given the following schema: {context}

    ### Response:
    {response}
    ### End"""
  else:
    # change instuction according to the task
    template = """### Instruction:
    You are a powerful text-to-SQL model. \
    Your job is to answer questions about a database. \
    You are given a question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    {question}
    ### Context:
    {context}
    ### Response:
    {response}
    ### End"""
  # print('Got Here!!!')
  dsdf['text'] = dsdf.apply(template.format_map, axis=1)
  display(dsdf.head(2))
  # dataset = Dataset.from_pandas(pd.DataFrame(dsdf['text'])).train_test_split(test_size=0.2)
  # dataset = Dataset.from_pandas(dsdf.loc[:, ['text']]).train_test_split(test_size=0.2)
  dataset = Dataset.from_pandas(dsdf).train_test_split(test_size=sample_rate,
                                                       seed=2025)
  print('Training Sample:')
  display(pd.DataFrame(dataset["train"]).head(2))
  print('Testing Sample:')
  display(pd.DataFrame(dataset["test"]).head(2))
  return dsdf, dataset



In [7]:
def process_test(ds, col='test', table_names=False, simple_inst=False):
  dsdf = pd.DataFrame(ds[col])
  # display(dsdf.head(2))

  # 2. Cleanup Steps
  # Dropping all examples where no definite table name is given
  # i.e. most of WikiSQL
  if table_names:
    dsdf = dsdf.loc[~dsdf['response'].str.contains('FROM table_')]
  # Drop Dups
  dsdf.drop_duplicates(inplace=True)
  # Drop "text" column since creating new "text" w/o Response
  dsdf.drop(columns=['text'], inplace=True)

  # 3. Simple Instruction
  if simple_inst:
    template = """Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    Generate SQL query: {question}, \
    given the following schema: {context}

    ### Response:
    """
  else:
    # change instuction according to the task
    template = """### Instruction:
    You are a powerful text-to-SQL model. \
    Your job is to answer questions about a database. \
    You are given a question and context regarding one or more tables.

    You must output the SQL query that answers the question.

    ### Input:
    {question}
    ### Context:
    {context}
    ### Response:
    """
  # print('Got Here!!!')
  dsdf['text'] = dsdf.apply(template.format_map, axis=1)
  display(dsdf.head(2))
  return dsdf

### Load and Store Process Dataset
- Stored as HF Dataset

In [8]:
# Note: table_names & simple_inst need to match in process_test and process_data
dsdf, dataset = process_data(dataset_name, sample_rate=SAMPLE_RATE,
                             table_names=TABLE_NAMES, simple_inst=SIMPLE_INST)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

,response,question,context,text
0,SELECT COUNT(*) FROM head WHERE age > 56,How many heads of the departments are older th...,CREATE TABLE head (age INTEGER),### Instruction:\n You are a powerful text-...
1,"SELECT name, born_state, age FROM head ORDER B...","List the name, born state and age of the heads...","CREATE TABLE head (name VARCHAR, born_state VA...",### Instruction:\n You are a powerful text-...


Training Sample:


,response,question,context,text,__index_level_0__
0,"SELECT Accreditation_type, COUNT(*) FROM phone...",Find the number of phones for each accreditati...,CREATE TABLE phone (Accreditation_type VARCHAR),### Instruction:\n You are a powerful text-...,579
1,SELECT COUNT(*) FROM game WHERE NOT id IN (SEL...,How many games are free of injury accidents?,"CREATE TABLE injury_accident (id VARCHAR, game...",### Instruction:\n You are a powerful text-...,731


Testing Sample:


,response,question,context,text,__index_level_0__
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),### Instruction:\n You are a powerful text-...,611
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",### Instruction:\n You are a powerful text-...,2556


In [9]:
dataset.save_to_disk(DATA_PATH + DS_DIR)

Saving the dataset (0/1 shards):   0%|          | 0/4086 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/454 [00:00<?, ? examples/s]

In [10]:
ds2=load_from_disk(DATA_PATH + DS_DIR)

#### Check Stored Dataset

In [11]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [12]:
display(ds2['train'])
display(ds2['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

### Save Test Pandas DataFrame

In [13]:
# Note: table_names & simple_inst need to match in process_test and process_data
test_df = process_test(dataset, col='test', table_names=TABLE_NAMES,
                       simple_inst=SIMPLE_INST)
display(test_df['text'][4])

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...


"### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    List all female students age is older than 18 who is not majoring in 600. List students' first name and last name.\n    ### Context:\n    CREATE TABLE Student (Fname VARCHAR, Lname VARCHAR, Sex VARCHAR, Age VARCHAR, Major VARCHAR)\n    ### Response:\n    "

In [15]:

# Ensure all directories exist
os.makedirs(DATA_PATH + PKL_DIR, exist_ok=True)
test_df.to_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [16]:
test_df2=pd.read_pickle(DATA_PATH+'test/'+PKL_FILE)

#### Check Stored Test DataFrame

In [17]:
display(test_df.head(2))

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...


In [18]:
display(test_df2.head(2))

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...
